In [ ]:
import pandas as pd
import re

#read raw datasets and preprocess them
df1 = pd.read_pickle("data/raw_data/antivaxtweetsfr_p1.pkl")
df2 = pd.read_pickle("data/raw_data/antivaxtweetsfr_p2.pkl")
df3 = pd.read_pickle("data/raw_data/antivaxtweetsfr_p3.pkl")
df4 = pd.read_pickle("data/raw_data/antivaxtweetsfr_p4.pkl")

In [ ]:
df3['created_at'] = pd.to_datetime(df3['created_at'])
df4['created_at'] = pd.to_datetime(df4['created_at'])

#Remove the columns that we are not using and that are not commun between datasets
df1 = df1.drop("impression_count", axis=1)
df2 = df2.drop("impression_count", axis=1)
df3 = df3.drop(["geo_information", "author_username", "author_followers_count", "author_tweet_count", "tweet_id", "author_tweet_count"], axis=1)
df3 = df3.rename(columns={"author id": "author_id"})
df4 = df4.drop(["geo_information", "author_username", "author_followers_count", "author_tweet_count", "tweet_id", "author_tweet_count"], axis=1)
df4 = df4.rename(columns={"author id": "author_id"})

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

#remove retweets
df = df[~df.tweet.str.startswith(('RT'))]

print(len(df))

403759


In [ ]:
def clean(text):
    #remove @ppl, url
    output = re.sub(r'https://\S*','', text)
    output = re.sub(r'@\S*','',output)

    #remove \r, \n
    rep = r'|'.join((r'\r',r'\n'))
    output = re.sub(rep,'',output)

    #remove duplicated punctuation
    output = re.sub(r'([!()\-{};:,<>./?@#$%\^&*_~]){2,}', lambda x: x.group()[0], output)

    #remove extra space
    output = re.sub(r'\s+', ' ', output).strip()

    #remove string if string only contains punctuation
    if sum([i.isalpha() for i in output])== 0:
        output = ''

    #remove string if length<5
    # Count the number of words without hashtags
    # num_words_without_hashtags = sum(1 for word in output.split() if not word.startswith('#'))

    # if num_words_without_hashtags < 5:
    #     output = ''
    return output

df['tweet_clean0'] = df['tweet'].apply(clean)

In [ ]:
#count tweets that violate Twitter policies (the tweets have been deleted and replaces by a message). As tweets are in French, the word "violates" is sufficient to find these tweets.
tweets_violates = df.loc[df['tweet_clean0'].str.contains('violates'), 'tweet_clean0']
print(len(tweets_violates))

1772


In [ ]:
#remove the tweets that violate Twitter policies
df.loc[df['tweet_clean0'].str.contains('violates'), 'tweet_clean0'] = ''
df = df[df['tweet_clean0'] != '']
#df = df[df['tweet_clean0'].apply(lambda tweet: not all(tag.startswith('#') for tag in tweet.split()))]

In [ ]:
len(df)

401987

In [ ]:
len(df.author_id.unique())

38836

In [ ]:
df.to_pickle("data/df_clean.pkl")

In [ ]:
df[df['tweet_clean0'].isnull()]

,author_id,created_at,retweet_count,reply_count,like_count,quote_count,lang,tweet,tweet_clean0
